# Compare Multiple Runs on Colab

This notebook allows you to compare:
- Different models (e.g., gf-6L-10M vs gf-12L-38M)
- Different optimization methods (baseline, batching, mixed precision, ONNX, TensorRT)
- Different configurations

The notebook imports both `GeneformerISPOptimizer` (baseline) and `OptimizedGeneformerISPOptimizer` (with optimizations) from the Python modules.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from baseline_isp import GeneformerISPOptimizer
from optimized_isp import OptimizedGeneformerISPOptimizer
import warnings
warnings.filterwarnings("ignore")

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 6)

print("Imports successful!")
print("Available classes:")
print("  - GeneformerISPOptimizer (baseline)")
print("  - OptimizedGeneformerISPOptimizer (with optimizations)")


## Configuration Options

You can compare:
1. **Different models** (e.g., gf-6L-10M vs gf-12L-38M)
2. **Different optimization methods** (baseline, batching, mixed precision, ONNX, TensorRT)
3. **Different batch sizes**

Choose your comparison type below:


In [ ]:
# Define configurations to compare
# You can compare different models OR different optimization methods

# Option 1: Compare different models (using baseline method)
configurations_models = [
    {
        'name': 'baseline_gf-6L-10M',
        'model_name': 'gf-6L-10M-i2048',
        'num_perturbations': 1000,
        'device': 'cuda' if torch.cuda.is_available() else 'cpu',
        'use_wandb': False,
        'optimizer_class': 'baseline',  # Use baseline optimizer
        'method': 'baseline'
    },
    # Uncomment to add more models:
    # {
    #     'name': 'baseline_gf-12L-38M',
    #     'model_name': 'gf-12L-38M-i4096',
    #     'num_perturbations': 1000,
    #     'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    #     'use_wandb': False,
    #     'optimizer_class': 'baseline',
    #     'method': 'baseline'
    # },
]

# Option 2: Compare different optimization methods (same model)
configurations_optimizations = [
    {
        'name': 'baseline',
        'model_name': 'gf-6L-10M-i2048',
        'num_perturbations': 1000,
        'device': 'cuda' if torch.cuda.is_available() else 'cpu',
        'use_wandb': False,
        'optimizer_class': 'baseline',
        'method': 'baseline',
        'batch_size': 10
    },
    {
        'name': 'batching_bs32',
        'model_name': 'gf-6L-10M-i2048',
        'num_perturbations': 1000,
        'device': 'cuda' if torch.cuda.is_available() else 'cpu',
        'use_wandb': False,
        'optimizer_class': 'optimized',
        'method': 'batching',
        'batch_size': 32
    },
    {
        'name': 'mixed_precision',
        'model_name': 'gf-6L-10M-i2048',
        'num_perturbations': 1000,
        'device': 'cuda' if torch.cuda.is_available() else 'cpu',
        'use_wandb': False,
        'optimizer_class': 'optimized',
        'method': 'mixed_precision',
        'batch_size': 32
    },
    # Uncomment if ONNX is available:
    {
        'name': 'onnx_runtime',
        'model_name': 'gf-6L-10M-i2048',
        'num_perturbations': 1000,
        'device': 'cuda' if torch.cuda.is_available() else 'cpu',
        'use_wandb': False,
        'optimizer_class': 'optimized',
        'method': 'onnx',
        'batch_size': 32
    },
]

# Choose which comparison to run
COMPARE_MODELS = False  # Set to True to compare different models
COMPARE_OPTIMIZATIONS = True  # Set to True to compare optimization methods

if COMPARE_MODELS:
    configurations = configurations_models
    print(f"Comparing different models: {len(configurations)} configurations")
elif COMPARE_OPTIMIZATIONS:
    configurations = configurations_optimizations
    print(f"Comparing optimization methods: {len(configurations)} configurations")
else:
    configurations = []
    print("No comparison selected. Set COMPARE_MODELS or COMPARE_OPTIMIZATIONS to True")


## Run All Configurations


In [ ]:
import os

all_results = []

for config in configurations:
    print(f"\n{'='*60}")
    print(f"Running: {config['name']}")
    print(f"{'='*60}")
    
    # Initialize optimizer based on configuration
    if config.get('optimizer_class') == 'optimized':
        optimizer = OptimizedGeneformerISPOptimizer(
            model_name=config['model_name'],
            device=config['device']
        )
    else:
        optimizer = GeneformerISPOptimizer(
            model_name=config['model_name'],
            device=config['device'],
            use_wandb=config.get('use_wandb', False),
            wandb_project='ispo-colab-comparison',
            wandb_run_name=config['name']
        )
    
    # Load model
    optimizer.load_model()
    
    # Load data
    data = optimizer.load_perturbation_data(num_perturbations=config['num_perturbations'])
    
    # Run inference based on method
    output_dir = f"results/{config['name']}"
    method = config.get('method', 'baseline')
    batch_size = config.get('batch_size', 10)
    
    if method == 'baseline':
        results = optimizer.run_baseline_inference(data, output_dir=output_dir)
    elif method == 'batching':
        if hasattr(optimizer, 'run_batching_optimized_inference'):
            results = optimizer.run_batching_optimized_inference(
                data, batch_size=batch_size, output_dir=output_dir
            )
        else:
            print(f"WARNING: Batching method not available, using baseline")
            results = optimizer.run_baseline_inference(data, output_dir=output_dir)
    elif method == 'mixed_precision':
        if hasattr(optimizer, 'run_mixed_precision_inference'):
            results = optimizer.run_mixed_precision_inference(
                data, batch_size=batch_size, output_dir=output_dir
            )
        else:
            print(f"WARNING: Mixed precision method not available, using baseline")
            results = optimizer.run_baseline_inference(data, output_dir=output_dir)
    elif method == 'onnx':
        if hasattr(optimizer, 'run_onnx_inference'):
            # First export to ONNX if needed
            onnx_path = f"model_{config['name']}.onnx"
            if not os.path.exists(onnx_path):
                optimizer.export_to_onnx(onnx_path)
            results = optimizer.run_onnx_inference(
                data, onnx_path, batch_size=batch_size, output_dir=output_dir
            )
        else:
            print(f"WARNING: ONNX not available, using baseline")
            results = optimizer.run_baseline_inference(data, output_dir=output_dir)
    else:
        # Default to baseline
        results = optimizer.run_baseline_inference(data, output_dir=output_dir)
    
    # Add configuration name
    results['config_name'] = config['name']
    results['method'] = config.get('method', 'baseline')
    
    all_results.append(results)
    
    # Finish wandb if used
    if config.get('use_wandb') and hasattr(optimizer, 'wandb_run') and optimizer.wandb_run:
        optimizer.wandb_run.finish()
    
    print(f"Completed: {config['name']}")

print(f"\nAll {len(all_results)} configurations completed!")


In [ ]:
# Create comparison summary
GeneformerISPOptimizer.save_results_summary(all_results, "results/comparison_summary.csv")

# Load and display
comparison_df = pd.read_csv("results/comparison_summary.csv")
print("Comparison Summary:")
print(f"Total methods compared: {len(comparison_df)}")
display(comparison_df)

# Show key metrics
if len(comparison_df) > 0:
    print("\nKey Performance Metrics:")
    if 'throughput_samples_per_sec' in comparison_df.columns:
        print(f"\nThroughput (samples/second):")
        for idx, row in comparison_df.iterrows():
            method = row.get('method', 'unknown')
            throughput = row.get('throughput_samples_per_sec', 0)
            if pd.notna(throughput):
                print(f"  {method:20s}: {throughput:.2f} samples/s")
    
    if 'zero_shot_accuracy' in comparison_df.columns:
        print(f"\nZero-shot Accuracy:")
        for idx, row in comparison_df.iterrows():
            method = row.get('method', 'unknown')
            accuracy = row.get('zero_shot_accuracy', 0)
            if pd.notna(accuracy):
                print(f"  {method:20s}: {accuracy:.4f}")


## Visualization Comparison


In [ ]:
# Create comparison plots
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Throughput comparison
if 'throughput_samples_per_sec' in comparison_df.columns:
    comparison_df.plot(x='method', y='throughput_samples_per_sec', 
                       kind='bar', ax=axes[0, 0], color='steelblue')
    axes[0, 0].set_title('Throughput Comparison', fontsize=14, fontweight='bold')
    axes[0, 0].set_ylabel('Samples/second')
    axes[0, 0].set_xlabel('')
    axes[0, 0].tick_params(axis='x', rotation=45)
    axes[0, 0].grid(True, alpha=0.3, axis='y')

# 2. Accuracy comparison
if 'zero_shot_accuracy' in comparison_df.columns:
    comparison_df.plot(x='method', y='zero_shot_accuracy',
                       kind='bar', ax=axes[0, 1], color='coral')
    axes[0, 1].set_title('Zero-Shot Accuracy Comparison', fontsize=14, fontweight='bold')
    axes[0, 1].set_ylabel('Accuracy')
    axes[0, 1].set_ylim(0, 1)
    axes[0, 1].set_xlabel('')
    axes[0, 1].tick_params(axis='x', rotation=45)
    axes[0, 1].grid(True, alpha=0.3, axis='y')

# 3. Silhouette score comparison
if 'silhouette_score' in comparison_df.columns:
    comparison_df.plot(x='method', y='silhouette_score',
                       kind='bar', ax=axes[1, 0], color='mediumseagreen')
    axes[1, 0].set_title('Silhouette Score Comparison', fontsize=14, fontweight='bold')
    axes[1, 0].set_ylabel('Silhouette Score')
    axes[1, 0].set_xlabel('')
    axes[1, 0].tick_params(axis='x', rotation=45)
    axes[1, 0].grid(True, alpha=0.3, axis='y')

# 4. Separation ratio comparison
if 'separation_ratio' in comparison_df.columns:
    comparison_df.plot(x='method', y='separation_ratio',
                       kind='bar', ax=axes[1, 1], color='gold')
    axes[1, 1].set_title('Separation Ratio Comparison', fontsize=14, fontweight='bold')
    axes[1, 1].set_ylabel('Separation Ratio')
    axes[1, 1].set_xlabel('')
    axes[1, 1].tick_params(axis='x', rotation=45)
    axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()


## Download Comparison Results


In [ ]:
# Download comparison results
from google.colab import files
import zipfile
import os

zip_path = 'comparison_results.zip'
with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    if os.path.exists('results/comparison_summary.csv'):
        zipf.write('results/comparison_summary.csv')
    for result in all_results:
        config_name = result['config_name']
        results_dir = f'results/{config_name}'
        if os.path.exists(results_dir):
            for root, dirs, files_list in os.walk(results_dir):
                for file in files_list:
                    file_path = os.path.join(root, file)
                    zipf.write(file_path, f'{config_name}/{os.path.relpath(file_path, results_dir)}')

files.download(zip_path)
print("✅ Comparison results downloaded!")
